The Nobel Prize has been among the most prestigious international awards since 1901. Each year, awards are bestowed in chemistry, literature, physics, physiology or medicine, economics, and peace. In addition to the honor, prestige, and substantial prize money, the recipient also gets a gold medal with an image of Alfred Nobel (1833 - 1896), who established the prize.

![](Nobel_Prize.png)

The Nobel Foundation has made a dataset available of all prize winners from the outset of the awards from 1901 to 2023. The dataset used in this project is from the Nobel Prize API and is available in the `nobel.csv` file in the `data` folder.

In this project, you'll get a chance to explore and answer several questions related to this prizewinning data. And we encourage you then to explore further questions that you're interested in!

In [40]:
# Loading in required libraries
import pandas as pd
import seaborn as sns
import numpy as np

# Start coding here!

First, we import our dataset as a DataFrame.

In [41]:
nobel = pd.read_csv("data/nobel.csv")
nobel.head()

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France


We can easily get find the most frequent gender and country by getting the frequency of the unique values in each attribute. Then, we can just get the first element in the returned Series.

In [42]:
top_gender = nobel["sex"].value_counts().index[0]
top_gender

'Male'

In [43]:
top_country = nobel["birth_country"].value_counts().index[0]
top_country

'United States of America'

Then, to get the decade with the highest ratio of nobels awarded to the USA, we first need to know the decade for each award. So we have to create a column that has the decade.

In [44]:
decade = []

for i, row in nobel.iterrows():
    yr = str(row["year"])[0:3] + "0"
    decade.append(int(yr))
    
nobel["decade"] = decade
nobel.head()

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country,decade
0,1901,Chemistry,The Nobel Prize in Chemistry 1901,"""in recognition of the extraordinary services ...",1/1,160,Individual,Jacobus Henricus van 't Hoff,1852-08-30,Rotterdam,Netherlands,Male,Berlin University,Berlin,Germany,1911-03-01,Berlin,Germany,1900
1,1901,Literature,The Nobel Prize in Literature 1901,"""in special recognition of his poetic composit...",1/1,569,Individual,Sully Prudhomme,1839-03-16,Paris,France,Male,NaN,NaN,NaN,1907-09-07,Châtenay,France,1900
2,1901,Medicine,The Nobel Prize in Physiology or Medicine 1901,"""for his work on serum therapy, especially its...",1/1,293,Individual,Emil Adolf von Behring,1854-03-15,Hansdorf (Lawice),Prussia (Poland),Male,Marburg University,Marburg,Germany,1917-03-31,Marburg,Germany,1900
3,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,462,Individual,Jean Henry Dunant,1828-05-08,Geneva,Switzerland,Male,NaN,NaN,NaN,1910-10-30,Heiden,Switzerland,1900
4,1901,Peace,The Nobel Peace Prize 1901,NaN,1/2,463,Individual,Frédéric Passy,1822-05-20,Paris,France,Male,NaN,NaN,NaN,1912-06-12,Paris,France,1900


We then get the frequencies for each decade.

In [45]:
decade_totals = pd.DataFrame(nobel.groupby("decade").count()["year"])
decade_totals.head()

,year
decade,
1900,57
1910,40
1920,54
1930,56
1940,43


We can now do those exact same steps but only to the data filtered by country to only nobels won by the USA.

In [46]:
decade_usa = pd.DataFrame(nobel[nobel["birth_country"] == "United States of America"].groupby("decade").count()["year"])
decade_usa.head()

,year
decade,
1900,1
1910,3
1920,4
1930,14
1940,13


We can now combine both sets by their decade and generate a new column that contains the ratio of USA awards won.

In [47]:
full_decade_usa = decade_totals
full_decade_usa["usa"] = decade_usa["year"]
full_decade_usa["ratio"] = full_decade_usa["usa"] / full_decade_usa["year"]
full_decade_usa = full_decade_usa.sort_values("ratio", ascending=False)
full_decade_usa.head()

,year,usa,ratio
decade,,,
2000,123,52,0.422764
1990,104,42,0.403846
2020,50,18,0.360000
1980,97,31,0.319588
1970,104,33,0.317308


As we can see, the decade with the biggest proportion of USA nobels was the 2000s.

In [48]:
max_decade_usa = 2000

Now, we can make a similar process to get the decade and the category with the biggest proportion of female awards. First, we get the frequencies for each combination of decade and category.

In [49]:
total_winners = pd.DataFrame(nobel.groupby(["decade", "category"]).count()["year"].sort_values(ascending=False))
total_winners = total_winners.sort_index()
total_winners.head()

year
decade category        
1900   Chemistry      9
       Literature    10
       Medicine      11
       Peace         14
       Physics       13

We now do the exact same thing, but only with the female awards.

In [50]:
total_females = pd.DataFrame(nobel[nobel["sex"] == "Female"].groupby(["decade", "category"]).count())["year"].sort_values(ascending=False)
total_females = total_females.sort_index()
total_females.head()

decade  category  
1900    Literature    1
        Peace         1
        Physics       1
1910    Chemistry     1
1920    Literature    2
Name: year, dtype: int64

We can now combine the results and calculate the ratio.

In [51]:
total_win_females = pd.merge(total_winners, total_females, how="left", on=["decade", "category"]).fillna(0)
total_win_females.head()

year_x  year_y
decade category                  
1900   Chemistry        9     0.0
       Literature      10     1.0
       Medicine        11     0.0
       Peace           14     1.0
       Physics         13     1.0

In [52]:
total_win_females["ratio"] = total_win_females["year_y"] / total_win_females["year_x"]
total_win_females = total_win_females.sort_values("ratio", ascending=False)
total_win_females.head()

,,year_x,year_y,ratio
decade,category,,,
2020,Literature,4,2.0,0.500000
2010,Peace,14,5.0,0.357143
1990,Literature,10,3.0,0.300000
2000,Literature,10,3.0,0.300000
2020,Chemistry,10,3.0,0.300000


We can extract the top row information to put it into the dictionary.

In [53]:
max_female = total_win_females.head(1)
max_female

,,year_x,year_y,ratio
decade,category,,,
2020,Literature,4,2.0,0.5


In [54]:
max_female_dict = {
    2020: "Literature"
}

max_female_dict

{2020: 'Literature'}

Now, to get the first award given to a woman, we can filter by sex and sort by year.

In [55]:
female = nobel[nobel["sex"] == "Female"].sort_values("year")
female.head()

,year,category,prize,motivation,prize_share,laureate_id,laureate_type,full_name,birth_date,birth_city,birth_country,sex,organization_name,organization_city,organization_country,death_date,death_city,death_country,decade
19,1903,Physics,The Nobel Prize in Physics 1903,"""in recognition of the extraordinary services ...",1/4,6,Individual,"Marie Curie, née Sklodowska",1867-11-07,Warsaw,Russian Empire (Poland),Female,NaN,NaN,NaN,1934-07-04,Sallanches,France,1900
29,1905,Peace,The Nobel Peace Prize 1905,NaN,1/1,468,Individual,"Baroness Bertha Sophie Felicita von Suttner, n...",1843-06-09,Prague,Austrian Empire (Czech Republic),Female,NaN,NaN,NaN,1914-06-21,Vienna,Austria,1900
51,1909,Literature,The Nobel Prize in Literature 1909,"""in appreciation of the lofty idealism, vivid ...",1/1,579,Individual,Selma Ottilia Lovisa Lagerlöf,1858-11-20,Mårbacka,Sweden,Female,NaN,NaN,NaN,1940-03-16,Mårbacka,Sweden,1900
62,1911,Chemistry,The Nobel Prize in Chemistry 1911,"""in recognition of her services to the advance...",1/1,6,Individual,"Marie Curie, née Sklodowska",1867-11-07,Warsaw,Russian Empire (Poland),Female,Sorbonne University,Paris,France,1934-07-04,Sallanches,France,1910
128,1926,Literature,The Nobel Prize in Literature 1926,"""for her idealistically inspired writings whic...",1/1,597,Individual,Grazia Deledda,1871-09-27,"Nuoro, Sardinia",Italy,Female,NaN,NaN,NaN,1936-08-15,Rome,Italy,1920


We see that Marie Curie was the first woman to recieve the Nobel prize in the category of physics.

In [56]:
first_woman_name = female.reset_index()["full_name"].loc[0]
first_woman_name

'Marie Curie, née Sklodowska'

In [57]:
first_woman_category = female.reset_index()["category"].loc[0]
first_woman_category

'Physics'

Finally, we can obtain the list of repeated winners by counting the frequencies of all the winners and filtering out all of those who have only 1 occurance.

In [58]:
nobel["name_or_org"] = nobel["organization_name"].fillna(nobel["full_name"])
repeat_df = pd.DataFrame(nobel.groupby("full_name")["year"].count().sort_values(ascending=False))
repeat_df = repeat_df[repeat_df["year"] != 1]
repeat_list = repeat_df.index.to_list()
repeat_list

['Comité international de la Croix Rouge (International Committee of the Red Cross)',
 'Office of the United Nations High Commissioner for Refugees (UNHCR)',
 'Frederick Sanger',
 'Linus Carl Pauling',
 'John Bardeen',
 'Marie Curie, née Sklodowska']